## Add elevation data to digitized buildings

In [1]:
import os

from multiprocessing import Pool
from scipy import stats

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import MultiPolygon

In [2]:
def mode(x):
    mode, count = stats.mode(np.round(x))
    return mode[0]

In [3]:
base_path = "../data/Digitization/"
elev_path = "../data/lidar/las_to_vector/"

Read all data

In [4]:
sample_names = [x.rstrip('_PointHeight_cor.gpkg') for x in os.listdir(elev_path) if x.endswith('_PointHeight_cor.gpkg')]
samples = {}
for sample in sample_names:
    print(f'Reading sample {sample}')
    digi_name = sample.replace('_', '') + '_Digitiz.shp'
    elev_name = sample + '_PointHeight_cor.gpkg'
    samples[sample] = {
        'digi': gpd.read_file(os.path.join(base_path, digi_name)),
        'elev': gpd.read_file(os.path.join(elev_path, elev_name))
    }

Reading sample FID_23
Reading sample FID_14
Reading sample FID_11
Reading sample FID_6
Reading sample FID_3
Reading sample FID_9
Reading sample FID_15
Reading sample FID_10
Reading sample FID_22
Reading sample FID_8
Reading sample FID_7
Reading sample FID_2
Reading sample FID_1
Reading sample FID_4
Reading sample FID_21
Reading sample FID_19
Reading sample FID_13
Reading sample FID_16
Reading sample FID_0
Reading sample FID_5
Reading sample FID_12
Reading sample FID_17
Reading sample FID_18
Reading sample FID_20


Normalize column names

In [8]:
for fid, dfs in samples.items():
    cols_to_rename = {
        'Id': 'OBJECTID',
        'Area': 'SHAPE_Area'
    }
    dfs['digi'].rename(index=str, columns=cols_to_rename, inplace=True)
    dfs['digi'] = dfs['digi'][['OBJECTID', 'SHAPE_Area', 'Roof', 'geometry']]

## Assign points to buildings

Then, aggregate data by building and join back to digitized DFs to save

In [11]:
for k, v in samples.items():
    print(f'Working on {k}')
    v['elev'] = v['elev'].to_crs(v['digi'].crs)
    df = gpd.sjoin(v['elev'], v['digi'], op='within')
    agg_df = df.groupby('index_right').agg(
        {'z': [min, max, np.mean, np.median, mode]}
    )
    agg_df.columns = agg_df.columns.get_level_values(1)
    
    # Check if multipolygons are present
    # and cast all simple ones to multi if necessary
    geom_types = v['digi'].geometry.geom_type.unique()
    if ('MultiPolygon' in geom_types) and (len(geom_types) > 1):
        print('Polygon and Multipolygons detected. Casting all to Multi...')
        v['digi'].geometry = v['digi'].geometry.apply(lambda x: MultiPolygon([x]))
    
    v['digi'].join(agg_df).to_file(
        os.path.join(os.path.join(base_path, 'output'), f'{k}_digi_elev.gpkg'),
        driver='GPKG'
    )

Working on FID_23
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_14
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_11
Working on FID_6
Working on FID_3
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_9
Working on FID_15
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_10
Working on FID_22
Working on FID_8
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_7
Working on FID_2
Working on FID_1
Working on FID_4
Working on FID_21
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_19
Working on FID_13
Working on FID_16
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_0
Working on FID_5
Working on FID_12
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_17
Working on FID_18
Polygon and Multipolygons detected. Casting all to Multi...
Working on FID_20


## Quickfix: Remove points from vector files where height is Out Of Bounds

In [3]:
elev_path = "../data/lidar/las_to_vector/"
def process_save(file):
    print(file)
    fp = os.path.join(elev_path, file)
    gdf = gpd.read_file(fp)
    gdf.loc[(gdf['z'] >= 0) & (gdf['z'] <= 25), :].to_file(fp, driver='GPKG')

In [4]:
pool = Pool()
files = [x for x in os.listdir(elev_path) if x.endswith('.gpkg')]
pool.map(process_save, files)

FID_20_PointHeight.gpkg
FID_15_PointHeight.gpkg
FID_10_PointHeight.gpkg
FID_1_PointHeight.gpkg
FID_11_PointHeight.gpkg
FID_21_PointHeight.gpkg
FID_0_PointHeight.gpkg
FID_5_PointHeight.gpkg
FID_14_PointHeight.gpkg
FID_4_PointHeight.gpkg
FID_13_PointHeight.gpkg
FID_18_PointHeight.gpkg
FID_8_PointHeight.gpkg
FID_3_PointHeight.gpkg
FID_7_PointHeight.gpkg
FID_17_PointHeight.gpkg
FID_12_PointHeight.gpkg
FID_9_PointHeight.gpkg
FID_19_PointHeight.gpkg
FID_6_PointHeight.gpkg
FID_23_PointHeight.gpkg
FID_22_PointHeight.gpkg
FID_2_PointHeight.gpkg
FID_16_PointHeight.gpkg


RuntimeError: Failed to write record: {'id': '92124', 'type': 'Feature', 'properties': {'new_class': 5, 'raw_class': 5, 'x': 779396.11, 'y': 2051605.49, 'z': 3.0}, 'geometry': {'type': 'Point', 'coordinates': (-72.3534009879044, 18.536586170133795)}}